# word embedding

In [0]:
import tensorflow as tf
print(tf.__version__)


1.15.0


In [0]:
tf.enable_eager_execution()

In [0]:
import tensorflow_datasets as tfds

In [0]:
imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXB2SWV/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXB2SWV/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXB2SWV/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

In [0]:
train_data, test_data = imdb['train'], imdb['test']

In [0]:
training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]


In [0]:
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

In [0]:
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [0]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type= "post"
oov_tok = "<OOV>"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from  tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [0]:
testing_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [0]:
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,embedding_dim, input_length=max_length),
      tf.keras.layers.GlobalAveragePooling1D(),
      tf.keras.layers.Dense(6,activation='relu'),
      tf.keras.layers.Dense(1,activation='sigmoid')
])

In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 102       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded,training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded,testing_labels_final)
          )

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


25000/25000 [==============================] - 6s 237us/sample - loss: 0.5609 - acc: 0.7432 - val_loss: 0.9541 - val_acc: 0.4983
Epoch 2/10
25000/25000 [==============================] - 5s 198us/sample - loss: 0.3426 - acc: 0.8596 - val_loss: 1.2938 - val_acc: 0.4966
Epoch 3/10
25000/25000 [==============================] - 5s 200us/sample - loss: 0.2768 - acc: 0.8874 - val_loss: 1.4993 - val_acc: 0.4980
Epoch 4/10
25000/25000 [==============================] - 5s 199us/sample - loss: 0.2403 - acc: 0.9068 - val_loss: 1.6685 - val_acc: 0.4968
Epoch 5/10
25000/25000 [==============================] - 5s 199us/sample - loss: 0.2140 - acc: 0.9182 - val_loss: 1.8246 - val_acc: 0.4967
Epoch 6/10
25000/25000 [==============================] - 5s 203us/sample - loss: 0.1933 - acc: 0.9289 - val_loss: 1.9417 - val_acc: 0.4956
Epoch 7/10
25000/25000 [==============================] - 5s 194us/sample - loss: 0.1749 - acc: 0.9381 - val_loss: 2.0593 - val_acc: 0.4946
Epoch 8/10
25000/25000 [=======

In [0]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
reverse_word_index = dict([ (value,key) for (key,value) in word_index.items()])

In [0]:
import io

In [0]:
out_v =io.open('vecs.tsv','w',encoding='utf-8')
out_m =io.open('meta.tsv','w',encoding='utf-8')

In [0]:
for word_num in range(1,vocab_size):
  word=reverse_word_index[word_num]
  embeddings=weights[word_num]
  out_m.write(word+'\n')
  out_v.write('\t'.join([str(x) for  x in embeddings])+'\n')

out_v.close()
out_m.close()

In [0]:
from google.colab import files

In [0]:
"""files.download('vecs.tsv')
files.download('meta.tsv')"""

"files.download('vecs.tsv')\nfiles.download('meta.tsv')"

In [0]:
predict_sent="The extremely real family situation, a touch of satire here and there, witty exchanges, everyday yet profound dialogues and excellent acting make the film strong in almost all areas. The best part is that it doesn't necessarily sound verbose with all the constant exchange of dialogues between the characters. Instead, the continuous flow of communication is good for the script. The depiction seems real without use of too much imagery. The director has kept it simple and straightforward yet heart-touching without going the poetic way. Each character has been created so beautifully that they seem real. There are no good or bad characters and above all, nothing in the script that talks of a highly principled central character. Borunbabu's family members and the extended family appear with all their human follies and yet, are dutiful. There flaws are neither glorified nor ignored. In fact, Borunbabu's flaws, as small as they are, also get noticed through his friend Sukumar's voice. The interpersonal relationships in the film are also unique and the director does a very good job of giving each character a distinct identity throughout the runtime of the film. There are one-on-one interactions on a multiple level and it's handled well."

In [0]:
predict = Tokenizer(num_words= vocab_size, oov_token= oov_tok)
predict.fit_on_texts(predict_sent)
word_index = tokenizer.word_index
sequences = predict.texts_to_sequences(training_sentences)
paddedpred = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [0]:
paddedpred.shape

(25000, 120)

In [0]:
f=model.predict(paddedpred)

In [0]:
y_classes = f.argmax(axis=1)

In [0]:
sum(y_classes)

0